Мета-информация в отдельной таблице:

    1 колонка - путь к файлу
    2 колонка - ресурс
    3 колонка - дата
    4 колонка - заголовок
    5 колонка - подзаголовок
    6 колонка - автор
    7 колонка - рубрика
    8 колонка - тэги(?)


In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
column_names = ['path', 'source', 'url', 'date', 'title', 'subtitle', 'author', 'rubrics', 'tags', 'genre']

# Nplus1 Parsing BeautifulSoup #

In [3]:
import re
from bs4 import BeautifulSoup

In [35]:
def parse_metadata_nplus1(file_paths: list, yandex_disk_names: list):
    ready_df = pd.DataFrame(columns=column_names)
    directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/'
    
    #pbar = tqdm(total=len(file_paths))
    #pbar = tqdm(total=100)
    
# Iterate through file_path, yd_name
    iterable = zip(file_paths, yandex_disk_names)
    
    for file_path, yandex_disk_name in tqdm(iterable):
       # pbar.update(100)
        row = []
        
        directory = directory_index + file_path.rstrip()
        
# Open text from source file
        with open(directory) as file_:
            source = file_.read()
            
    # Compile path
        path_index_yd = 'nplus1.ru/nplus1_blog/'
        path = path_index_yd + yandex_disk_name
        row.append(path)
        
    # Add source
        web_source = 'nplus1.ru'
        row.append(web_source)

    # Compile url 
        re_link = re.compile('data\/(.*)\.html')
        link = re_link.findall(file_path)
        link = ''.join(link) 
        row.append(link)

        
        
        soup_article = BeautifulSoup(source, 'html.parser')
        
    # Find date
        raw_date = soup_article.find(attrs={'itemprop': 'datePublished'})['content']
        row.append(raw_date)
        

    # Find a title
        try:

            head_title = soup_article.find('h1').text.strip()
            
        except AttributeError:
            head_title = ''
        
        row.append(head_title)
        
    # Add subtitle        
        subtitle = ''
        row.append(subtitle)

    # Find an author
        try:

            re_author = re.compile('[A-Za-z]')
            author = soup_article.find(attrs={'name': 'mediator_author'})['content']
            if re_author.match(author):
                author = ''
       # author = ''
        except TypeError:
            author = ''
        
        row.append(author)
        
    # Find rubrics
        try:
            broad_rubrics = soup_article.find('p', class_='table').text 
            broad_rubrics = broad_rubrics.split('\n')
            broad_rubrics = '_'.join(broad_rubrics)
        except AttributeError:
            broad_rubrics = ''
        row.append(broad_rubrics)
        
    # Add tags
        rubrics = ''
        row.append(rubrics)

    # Add genre
        genre = 'Блоги'
        row.append(genre)

    # Compile data frame
        ready_df = ready_df.append(pd.Series(row, index=column_names), ignore_index=True)
        
        #pbar.close()
    return ready_df

# Nplus1 news #

In [31]:
# Read through indices
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_news_index.txt') as ps_index:
    read_index = ps_index.readlines() 

In [6]:
# Read through file names on YandexDisk
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/nplus1/yd_news.txt', 'r') as yd_n:
    yd_index = yd_n.readlines()

In [7]:
print('read-index:', len(read_index), '\nyd_index:', len(yd_index))

read-index: 11170 
yd_index: 11170


In [17]:
df_nplus1_news = parse_metadata_nplus1(read_index, yd_index)

A Jupyter Widget

In [19]:
df_nplus1_news.head(3)

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,nplus1.ru/nplus1_news/nplus1.ru-news-2015-03-0...,nplus1.ru,nplus1.ru/news/2015/03/02/bionics,2015-03-02,Травму нервного плечевого сплетения научились ...,,,_Наука_Разное_,,Новости
1,nplus1.ru/nplus1_news/nplus1.ru-news-2015-03-0...,nplus1.ru,nplus1.ru/news/2015/03/02/blackhole,2015-03-02,Рекордно молодая сверхмассивная дыра удивила ф...,,,_Космос_Наука_Разное_,,Новости
2,nplus1.ru/nplus1_news/nplus1.ru-news-2015-03-0...,nplus1.ru,nplus1.ru/news/2015/03/02/chaotic,2015-03-02,Физики поделили экономики на «ламинарные» и «х...,,,_Наука_Физики смогли_,,Новости


In [217]:
unique_rubrics = []
rubrics_list = df_nplus1_news['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [218]:
with open('nplus1_news_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [20]:
df_nplus1_news_csv = df_nplus1_news.to_csv()

In [21]:
with open('nplus1_news.csv', 'w') as fl:
    fl.write(df_nplus1_news_csv)
    

# Nplus1 materials #

In [22]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_material_index.txt') as ps_index:
    read_index = ps_index.readlines() 

In [23]:
# Read through file names on YandexDisk
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/nplus1/yd_materials.txt', 'r') as yd_n:
    yd_index = yd_n.readlines()

In [26]:
print('read-index:', len(read_index), '\nyd_index:', len(yd_index))

read-index: 536 
yd_index: 536


In [27]:
df_nplus1_materials = parse_metadata_nplus1(read_index, yd_index)

A Jupyter Widget

In [29]:
df_nplus1_materials.head(3)

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,nplus1.ru/nplus1_materials/nplus1.ru-material-...,nplus1.ru,nplus1.ru/material/2015/04/10/waranimals,2015-04-10,"Вы звери, господа",,Александр Ершов,_Наука_Археология_,,Статьи
1,nplus1.ru/nplus1_materials/nplus1.ru-material-...,nplus1.ru,nplus1.ru/material/2015/04/11/greencloud,2015-04-11,Зеленые рукава галактик,,Владимир Королев,_Космос_,,Статьи
2,nplus1.ru/nplus1_materials/nplus1.ru-material-...,nplus1.ru,nplus1.ru/material/2015/04/11/seasontwo,2015-04-11,Коллайдер: Перезагрузка,,Александр Ершов,_Наука_Второй сезон Коллайдера_,,Статьи


In [251]:
unique_rubrics = []
rubrics_list = df_nplus1_materials['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [253]:
with open('nplus1_materials_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [30]:
df_nplus1_materials_csv = df_nplus1_materials.to_csv()
with open('nplus1_materials.csv', 'w') as fl:
    fl.write(df_nplus1_materials_csv)

# Nplus1 Blog #

In [33]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_blog_index.txt') as ps_index:
    read_index = ps_index.readlines() 

In [32]:
# Read through file names on YandexDisk
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/nplus1/yd_blog.txt', 'r') as yd_n:
    yd_index = yd_n.readlines()

In [36]:
df_nplus1_blog = parse_metadata_nplus1(read_index, yd_index)

A Jupyter Widget

In [38]:
df_nplus1_blog.head(3)

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,nplus1.ru/nplus1_blog/nplus1.ru-blog-2015-03-2...,nplus1.ru,nplus1.ru/blog/2015/03/23/andromeda,2015-03-23,Непростое прошлое Андромеды,,Марат Мусин,_Космос_Астрономические дневники_,,Блоги
1,nplus1.ru/nplus1_blog/nplus1.ru-blog-2015-03-2...,nplus1.ru,nplus1.ru/blog/2015/03/27/trail,2015-03-27,По млечному следу,,Марат Мусин,_Блоги_Астрономические дневники_,,Блоги
2,nplus1.ru/nplus1_blog/nplus1.ru-blog-2015-04-0...,nplus1.ru,nplus1.ru/blog/2015/04/21/founding-of-rome,2015-04-21,21 апреля 753 года до нашей эры был основан Рим,,Тарас Молотилин,_Блоги_Блоги_День в истории науки_,,Блоги


In [329]:
unique_rubrics = []
rubrics_list = df_nplus1_blogs['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [330]:
with open('nplus1_blogs_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [40]:
df_nplus1_blogs_csv = df_nplus1_blog.to_csv()
with open('nplus1_blogs.csv', 'w') as fl:
    fl.write(df_nplus1_blogs_csv)

# ProScience Parsing #

In [48]:
import re
import bs4 as bs
import pandas as pd

In [102]:
def parse_metadata_polit(file_paths: list, yandex_disk_names: list):
    ready_df = pd.DataFrame(columns=column_names)
    directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/polit.ru_prosience_wget/polit.ru/'
    #pbar = tqdm(total=len(file_paths))
    #pbar = tqdm(total=100)
    
# Iterate through file_path, yd_name
    iterable = zip(file_paths, yandex_disk_names)
    
    for file_path, yandex_disk_name in tqdm(iterable):
       # pbar.update(100)
        row = []
        file_path = file_path[1:]
        directory = directory_index + file_path.rstrip()
        
# Open text from source file
        with open(directory) as file_:
            source = file_.read()
            
    # Compile path
        path_index_yd = 'polit.ru_proscience/proscience_news/'
        path = path_index_yd + yandex_disk_name
        row.append(path)
        
    # Add source
        web_source = 'http://polit.ru/rubric/proscience/'
        row.append(web_source)

    # Compile url 
        re_link = re.compile('.*/polit.ru_prosience_wget\/(.*)index\.html')
        link = re_link.match(directory)
        link = link.group(1)
        link = ''.join(link) 
        row.append(link)
        
        soup_article = bs.BeautifulSoup(source, 'lxml')
        
    # Find date
        raw_date = soup_article.find('div', class_='date')
        pre_date = re.search(r'.*([0-9]{2}\s[а-я]+\s[0-9]{4}).*', str(raw_date))
        date = pre_date.group(1)
        row.append(date)
        
        
    # Find a title
        head_title = soup_article.find('h1', class_='title').text
        
        row.append(head_title)
 
    # Add subtitle        
        subtitle = ''
        row.append(subtitle)

    # Find an author
        try:    
            author = soup_article.find('div', class_ = 'authors').text.strip()
        except AttributeError:
            author = ''
        
        row.append(author)

    # Find rubrics
        broad_rubrics = ''
        row.append(broad_rubrics)
        
    # Add tags
        rubrics = ''
        _rubrics = soup_article.find_all('div', class_= 'tags-subject')
        for rub_href in _rubrics:
            rub_tag = rub_href.select('a[href]')
            for tag in rub_tag:
                rubrics = rubrics + '_' + tag.get_text().lower()  
        row.append(rubrics)
    # Add genre
        genre = 'Новости'
        row.append(genre)

    # Compile data frame
        ready_df = ready_df.append(pd.Series(row, index=column_names), ignore_index=True)

        #pbar.close()
    return ready_df

In [49]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/polit.ru_prosience_wget/proscience_index.txt') as ps_index:
    read_index = ps_index.readlines()    

In [51]:
# Read through file names on YandexDisk
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/pro_science/for articles/yd_ps_articles.txt', 'r') as yd_n:
    yd_index = yd_n.readlines()

In [52]:
print('read-index:', len(read_index), '\nyd_index:', len(yd_index))

read-index: 1412 
yd_index: 1411


In [103]:
df_ps_articles = parse_metadata_polit(read_index, yd_index)

A Jupyter Widget

KeyboardInterrupt: 

In [90]:
df_ps_articles.head(3)

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,polit.ru_proscience/proscience_articles/articl...,http://polit.ru/rubric/proscience/,polit.ru/article/2012/10/01/ps_krill/,01 октября 2012,Антарктика: большая роль маленького криля,,Екатерина Урюпова,,_биология _глобальное потепление_естественные ...,Статьи
1,polit.ru_proscience/proscience_articles/articl...,http://polit.ru/rubric/proscience/,polit.ru/article/2012/10/11/ps_sc20_aleksandrov1/,11 октября 2012,"Трудовая миграция: мигрируют не люди, а сети",,,,_гуманитарные и социальные науки_даниил алекс...,Статьи
2,polit.ru_proscience/proscience_articles/articl...,http://polit.ru/rubric/proscience/,polit.ru/article/2012/10/12/ps_sc20_aleksandrov2/,12 октября 2012,Даниил Александров: «Крупные ученые изменяют р...,,,,_даниил александров_мегагранты_наука 2.0,Статьи


In [92]:
unique_tags = []
tags_list = df_ps_articles['tags'].values
tags_list = tags_list.tolist()
for line in tags_list:
    split_tags = line.split('_')
    split_tags = split_tags[1:]
    unique_tags.extend(split_tags)
unique_tags = set(unique_tags)
unique_tags = '\n'.join(unique_tags)

In [60]:
with open('proscience_articles_tag_list.txt', 'w') as fl:
    fl.write(unique_tags)

In [93]:
df_ps_articles.to_csv('proscience_articles.csv')

## ProScience News ##

In [94]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/pro_science/for news/proscience_index_news.txt') as ps_index:
    read_index = ps_index.readlines()    

In [96]:
# Read through file names on YandexDisk
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/pro_science/for news/yd_ps_news.txt', 'r') as yd_n:
    yd_index = yd_n.readlines()

In [97]:
print('read-index:', len(read_index), '\nyd_index:', len(yd_index))

read-index: 3849 
yd_index: 3849


In [107]:
df_ps_news = parse_metadata_polit(read_index, yd_index)

A Jupyter Widget

In [108]:
df_ps_news.head(3)

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,polit.ru_proscience/proscience_news/news-2012-...,http://polit.ru/rubric/proscience/,polit.ru//news/2012/09/12/ps_bacteriofag/,12 сентября 2012,"Бактериофаги можно купить только в Грузии, Рос...",,,,_бактериофаги_биология _естественные и точные ...,Новости
1,polit.ru_proscience/proscience_news/news-2012-...,http://polit.ru/rubric/proscience/,polit.ru//news/2012/09/12/ps_brain_education/,12 сентября 2012,Исследования мозга заставляют пересмотреть под...,,,,_естественные и точные науки_мозг_нейробиология,Новости
2,polit.ru_proscience/proscience_news/news-2012-...,http://polit.ru/rubric/proscience/,polit.ru//news/2012/09/13/ps_parasite_tounge/,13 сентября 2012,Мокрицы-паразиты умеют превращаться в рыбьи языки,,,,_биология _естественные и точные науки_паразиты,Новости


In [109]:
unique_tags = []
tags_list = df_ps_news['tags'].values
tags_list = tags_list.tolist()
for line in tags_list:
    split_tags = line.split('_')
    split_tags = split_tags[1:]
    unique_tags.extend(split_tags)
unique_tags = set(unique_tags)
unique_tags = '\n'.join(unique_tags)

In [67]:
with open('proscience_news_tag_list.txt', 'w') as fl:
    fl.write(unique_tags)

In [110]:
df_ps_news.to_csv('proscience_news.csv')